In [1]:
%config IPCompleter.greedy=True

In [2]:
from pyspark import SparkContext
import findspark as fs
import re
import random

In [3]:
fs.init()
sc = SparkContext(appName="Graphs")

## Exercise 31

In [4]:
# generate file with graph
vertices = random.randint(100, 1000)

with open('./data/graph.txt', 'w') as f:
    f.write('[\n')
    for v in range(1, vertices+1):
        neighbours = []
        neighbours_count = random.randint(1, 10)

        while len(neighbours) < neighbours_count:
            next = random.randint(1, vertices)
            
            if next not in neighbours:
                neighbours.append(next)
        
        neighbours.sort()
        
        f.write(str([v, neighbours]) + ',\n')
    
    f.write(']\n')

In [5]:
def get_edges(row):
    res = re.search(r'\[(\d+), \[([\d, ]+)\]\]', row)
    print(res)
    return (res.group(1), res.group(2).split(','))

In [6]:
edges = sc.textFile('./data/graph.txt').filter(lambda row: ',' in row).map(get_edges)
mapped = edges.flatMap(lambda pair: [(v, pair[0]) for v in pair[1]])
groupped = mapped.groupByKey().map(lambda pair: (pair[0], list(pair[1]))).sortByKey()
groupped.collect()[0:10]

[(' 100', ['11', '528']),
 (' 101', ['196', '387', '531']),
 (' 102', ['88', '387', '453', '536']),
 (' 103', ['11', '51', '116', '400', '613']),
 (' 104', ['119', '177', '254', '362', '536']),
 (' 105', ['305', '379']),
 (' 106', ['26', '63', '186', '247', '398']),
 (' 107', ['24', '224', '362', '434']),
 (' 108', ['88', '89', '512', '634']),
 (' 109', ['51', '475'])]

## Exercise 36

In [7]:
def map_vertices(row):
    splitted = row.split()
    return (splitted[0], splitted[1])

In [8]:
edges = sc.textFile('./data/stanford_graph.txt').map(map_vertices) # (v1 -> v2)

In [9]:
in_out_pairs = edges.flatMap(lambda pair: [(pair[0], 'out'), (pair[1], 'in')]) # [(v1, 'out'), (v2, 'in')]

In [10]:
counted_pairs = in_out_pairs.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b) # ((v, 'out/in'), 1) -> aggregate by key ((v, 'out/in'), m)

In [11]:
sorted_pairs = counted_pairs.sortBy(lambda pair: pair[0][1], ascending=True) # ['in', 'in', ..., 'out', 'out']

In [12]:
pairs_with_edge_as_key = sorted_pairs.map(lambda pair: (pair[0][0], (pair[0][1], pair[1]))) # (v, ('in/out', n))

In [13]:
grouped_pairs = pairs_with_edge_as_key.groupByKey() # (v, [('in', n), ('out', m)])

In [14]:
def get_triplet(pair):
    counted = list(pair[1])
    
    if len(counted) == 2:
        return (pair[0], counted[0][1], counted[1][1])
    elif counted[0][0] == 'in':
        return (pair[0], counted[0][1], 0)
    else:
        return (pair[0], 0, counted[0][1])

In [15]:
vertices_set = grouped_pairs.map(get_triplet) # (v, n, m)
vertices_set.take(5)

[('15409', 3, 1),
 ('17794', 2, 1),
 ('25202', 2, 1),
 ('53625', 2, 1),
 ('54582', 2, 1)]

In [16]:
mapped_vertices = vertices_set.map(lambda x: (x[1], 1))
mapped_vertices.reduce(lambda a, b: ((a[0] * a[1] + b[0] * b[1]) / (a[1] + b[1]), a[1] + b[1]))[0]

8.203165627893243

In [17]:
mapped_vertices = vertices_set.map(lambda x: (x[2], 1))
mapped_vertices.reduce(lambda a, b: ((a[0] * a[1] + b[0] * b[1]) / (a[1] + b[1]), a[1] + b[1]))[0]

8.2031656278933

# Exercise 37

In [18]:
data = sc.textFile('./data/stanford_graph.txt')

In [19]:
def to_pairs(line):
    vs = line.split('\t')
    v0 = int(vs[0])
    v1 = int(vs[1])
    
    return (
        [v0, [v1]],
        [v1, [v0]]
    )

In [20]:
graph = data.flatMap(to_pairs)
graph = graph.reduceByKey(lambda adjs1, adjs2: adjs1 + adjs2)
graph = graph.reduceByKey(set)
graph = graph.sortByKey()

In [21]:
adjs = graph.collectAsMap()

In [22]:
degs = graph.mapValues(len).collectAsMap()

In [23]:
br_adjs = sc.broadcast(adjs)

In [24]:
def calc_clustering_coeff(v, deg):
    denom = deg * (deg - 1.0)
    
    if denom == 0:
        return 0
    
    triangles = 0
    
    for u in br_adjs.value[v]:
        for w in br_adjs.value[v]:
            if w in br_adjs.value[u]:
                triangles += 1
    
    return triangles / denom

In [25]:
vertices = graph.keys()

In [26]:
result = vertices.map(lambda v: (v, calc_clustering_coeff(v, degs[v]), degs[v]))

In [27]:
result.take(10)

[(1, 0.0, 2),
 (2, 0.000594059405940594, 101),
 (3, 0.0, 2),
 (4, 0.6666666666666666, 4),
 (5, 0.8444444444444444, 10),
 (6, 0.0, 4),
 (7, 0.0, 7),
 (8, 0.8571428571428571, 7),
 (9, 1.0, 2),
 (10, 0.4, 5)]

In [28]:
coeffs = result.map(lambda triplet: triplet[1])
coeffs.take(10)

[0.0,
 0.000594059405940594,
 0.0,
 0.6666666666666666,
 0.8444444444444444,
 0.0,
 0.0,
 0.8571428571428571,
 1.0,
 0.4]

In [29]:
avg = coeffs.mean()

In [30]:
print(avg)

0.5976304608027554
